In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master('local[*]').appName('test').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/26 11:21:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option('header', 'true').parquet('../data/fhvhv_tripdata_2022-01.parquet')
#df.show()
df.head()

Row(hvfhs_license_num='HV0003', dispatching_base_num='B03404', originating_base_num='B03404', request_datetime=datetime.datetime(2022, 1, 1, 0, 5, 31), on_scene_datetime=datetime.datetime(2022, 1, 1, 0, 5, 40), pickup_datetime=datetime.datetime(2022, 1, 1, 0, 7, 24), dropoff_datetime=datetime.datetime(2022, 1, 1, 0, 18, 28), PULocationID=170, DOLocationID=161, trip_miles=1.18, trip_time=664, base_passenger_fare=24.9, tolls=0.0, bcf=0.75, sales_tax=2.21, congestion_surcharge=2.75, airport_fee=0.0, tips=0.0, driver_pay=23.03, shared_request_flag='N', shared_match_flag='N', access_a_ride_flag=' ', wav_request_flag='N', wav_match_flag='N')

In [4]:
df.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampType(), True), StructField('on_scene_datetime', TimestampType(), True), StructField('pickup_datetime', TimestampType(), True), StructField('dropoff_datetime', TimestampType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_request_flag',

In [5]:
#spark.createDataFrame(pandas_df) --> to create spark dataframe
'''
define a schema in spark
from pyspark.sql import types
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])
'''
df.repartition(24)

DataFrame[hvfhs_license_num: string, dispatching_base_num: string, originating_base_num: string, request_datetime: timestamp, on_scene_datetime: timestamp, pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: bigint, DOLocationID: bigint, trip_miles: double, trip_time: bigint, base_passenger_fare: double, tolls: double, bcf: double, sales_tax: double, congestion_surcharge: double, airport_fee: double, tips: double, driver_pay: double, shared_request_flag: string, shared_match_flag: string, access_a_ride_flag: string, wav_request_flag: string, wav_match_flag: string]

In [10]:
df.write.parquet('fhvhv_tripdata_2022-01_part')

In [6]:
df = spark.read.parquet('../data/fhvhv_tripdata_2022-01_part')

In [8]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- originating_base_num: string (nullable = true)
 |-- request_datetime: timestamp (nullable = true)
 |-- on_scene_datetime: timestamp (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- trip_miles: double (nullable = true)
 |-- trip_time: long (nullable = true)
 |-- base_passenger_fare: double (nullable = true)
 |-- tolls: double (nullable = true)
 |-- bcf: double (nullable = true)
 |-- sales_tax: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- airport_fee: double (nullable = true)
 |-- tips: double (nullable = true)
 |-- driver_pay: double (nullable = true)
 |-- shared_request_flag: string (nullable = true)
 |-- shared_match_flag: string (nullable = true)
 |-- access_a_ride_flag: string (nul

In [12]:
# selecting columns and filter for specific records....
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
  .filter(df['hvfhs_license_num'] == 'HV0003').show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2022-01-01 00:07:24|2022-01-01 00:18:28|         170|         161|
|2022-01-01 00:22:32|2022-01-01 00:30:12|         237|         161|
|2022-01-01 00:57:37|2022-01-01 01:07:32|         237|         161|
|2022-01-01 00:18:02|2022-01-01 00:23:05|         262|         229|
|2022-01-01 00:28:01|2022-01-01 00:35:42|         229|         141|
|2022-01-01 00:38:50|2022-01-01 00:51:32|         263|          79|
|2022-01-01 00:53:25|2022-01-01 01:08:56|         113|         140|
|2022-01-01 00:08:58|2022-01-01 00:23:01|         151|          75|
|2022-01-01 00:32:25|2022-01-01 00:44:15|         263|         229|
|2022-01-01 00:50:23|2022-01-01 01:15:30|         237|         169|
|2022-01-01 00:17:02|2022-01-01 00:40:09|         261|         223|
|2022-01-01 00:43:20|2022-01-01 00:47:31|       

In [14]:
from pyspark.sql import functions as F

In [20]:
# F.to_date() : convert given col to date...
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID')\
  .withColumn('pickup_date', F.to_date(df['pickup_datetime'])).show(5)

+-------------------+-------------------+------------+------------+-----------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|pickup_date|
+-------------------+-------------------+------------+------------+-----------+
|2022-01-01 00:07:24|2022-01-01 00:18:28|         170|         161| 2022-01-01|
|2022-01-01 00:22:32|2022-01-01 00:30:12|         237|         161| 2022-01-01|
|2022-01-01 00:57:37|2022-01-01 01:07:32|         237|         161| 2022-01-01|
|2022-01-01 00:18:02|2022-01-01 00:23:05|         262|         229| 2022-01-01|
|2022-01-01 00:28:01|2022-01-01 00:35:42|         229|         141| 2022-01-01|
+-------------------+-------------------+------------+------------+-----------+
only showing top 5 rows



In [25]:
# more complex function that u can't create in sql
def calc_time_hr(t):
    return t / 3600
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_time')\
  .withColumn('trip_time_hr', calc_time_hr(df['trip_time']))\
  .show(5)

+-------------------+-------------------+------------+------------+---------+-------------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_time|       trip_time_hr|
+-------------------+-------------------+------------+------------+---------+-------------------+
|2022-01-01 00:07:24|2022-01-01 00:18:28|         170|         161|      664|0.18444444444444444|
|2022-01-01 00:22:32|2022-01-01 00:30:12|         237|         161|      460|0.12777777777777777|
|2022-01-01 00:57:37|2022-01-01 01:07:32|         237|         161|      595|0.16527777777777777|
|2022-01-01 00:18:02|2022-01-01 00:23:05|         262|         229|      303|0.08416666666666667|
|2022-01-01 00:28:01|2022-01-01 00:35:42|         229|         141|      461|0.12805555555555556|
+-------------------+-------------------+------------+------------+---------+-------------------+
only showing top 5 rows

